In [ ]:
import re
import string
from unicodedata import normalize
import numpy
import os 

# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, mode='rt', encoding='utf-8')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text


# split a loaded document into sentences
def to_pairs(doc):
    lines = doc.strip().split('\n')
    pairs = [line.split('\t') for line in  lines]
    return pairs

def clean_data(lines):
    cleaned = list()
    # prepare regex for char filtering
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for pair in lines:
        clean_pair = list()
        for line in pair:
            # normalize unicode characters
            line = normalize('NFD', line).encode('ascii', 'ignore')
            line = line.decode('UTF-8')
            # tokenize on white space
            line = line.split()
            # convert to lowercase
            line = [word.lower() for word in line]
            # remove punctuation from each token
            line = [word.translate(table) for word in line]
            # remove non-printable chars form each token
            line = [re_print.sub('', w) for w in line]
            # remove tokens with numbers in them
            line = [word for word in line if word.isalpha()]
            # store as string
            clean_pair.append(' '.join(line))
        cleaned.append(clean_pair)
    return numpy.array(cleaned)

In [ ]:
filename = 'spa.txt'
n_train = 20000

In [ ]:
# load dataset
doc = load_doc(filename)

# split into Language1-Language2 pairs
pairs = to_pairs(doc)

# clean sentences
clean_pairs = clean_data(pairs)[0:n_train, :]

In [ ]:
for i in range(3000, 3010):
    print('[' + clean_pairs[i, 0] + '] => [' + clean_pairs[i, 1] + ']')

[youre here] => [estas aqui]
[youre here] => [estais aqui]
[youre late] => [estas retrasado]
[youre lost] => [estas perdido]
[youre mean] => [eres mala]
[youre mean] => [eres mezquino]
[youre mine] => [tu eres mio]
[youre nice] => [eres simpatico]
[youre nuts] => [estas loco]
[youre nuts] => [estas chiflado]


In [ ]:
input_texts = clean_pairs[:, 0]
target_texts = ['\t' + text + '\n' for text in clean_pairs[:, 1]]

print('Length of input_texts:  ' + str(input_texts.shape))
print('Length of target_texts: ' + str(input_texts.shape))

Length of input_texts:  (20000,)
Length of target_texts: (20000,)


In [ ]:
max_encoder_seq_length = max(len(line) for line in input_texts)
max_decoder_seq_length = max(len(line) for line in target_texts)

print('max length of input  sentences: %d' % (max_encoder_seq_length))
print('max length of target sentences: %d' % (max_decoder_seq_length))

max length of input  sentences: 18
max length of target sentences: 55


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# encode and pad sequences
def text2sequences(max_len, lines):
    tokenizer = Tokenizer(char_level=True, filters='')
    tokenizer.fit_on_texts(lines)
    seqs = tokenizer.texts_to_sequences(lines)
    seqs_pad = pad_sequences(seqs, maxlen=max_len, padding='post')
    return seqs_pad, tokenizer.word_index


encoder_input_seq, input_token_index = text2sequences(max_encoder_seq_length, 
                                                      input_texts)
decoder_input_seq, target_token_index = text2sequences(max_decoder_seq_length, 
                                                       target_texts)

print('shape of encoder_input_seq: ' + str(encoder_input_seq.shape))
print('shape of input_token_index: ' + str(len(input_token_index)))
print('shape of decoder_input_seq: ' + str(decoder_input_seq.shape))
print('shape of target_token_index: ' + str(len(target_token_index)))

shape of encoder_input_seq: (20000, 18)
shape of input_token_index: 27
shape of decoder_input_seq: (20000, 55)
shape of target_token_index: 29


In [ ]:
num_encoder_tokens = len(input_token_index) + 1
num_decoder_tokens = len(target_token_index) + 1

print('num_encoder_tokens: ' + str(num_encoder_tokens))
print('num_decoder_tokens: ' + str(num_decoder_tokens))

num_encoder_tokens: 28
num_decoder_tokens: 30


In [ ]:
target_texts[100]

'\tno puede ser\n'

In [ ]:
decoder_input_seq[100, :]

array([ 6,  8,  3,  1, 17, 14,  2, 15,  2,  1,  5,  2, 10,  7,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0], dtype=int32)

In [ ]:
from tensorflow.keras.utils import to_categorical

# one hot encode target sequence
def onehot_encode(sequences, max_len, vocab_size):
    n = len(sequences)
    data = numpy.zeros((n, max_len, vocab_size))
    for i in range(n):
        data[i, :, :] = to_categorical(sequences[i], num_classes=vocab_size)
    return data

encoder_input_data = onehot_encode(encoder_input_seq, max_encoder_seq_length, num_encoder_tokens)
decoder_input_data = onehot_encode(decoder_input_seq, max_decoder_seq_length, num_decoder_tokens)

decoder_target_seq = numpy.zeros(decoder_input_seq.shape)
decoder_target_seq[:, 0:-1] = decoder_input_seq[:, 1:]
decoder_target_data = onehot_encode(decoder_target_seq, 
                                    max_decoder_seq_length, 
                                    num_decoder_tokens)

print(encoder_input_data.shape)
print(decoder_input_data.shape)

(20000, 18, 28)
(20000, 55, 30)


In [ ]:
from keras.layers import Input, LSTM, Bidirectional, Concatenate, LSTM
from keras.models import Model

latent_dim = 64


encoder_inputs = Input(shape=(None, num_encoder_tokens), 
                       name='encoder_inputs')


encoder_lstm = Bidirectional(LSTM(latent_dim , return_state=True, 
                    dropout=0.5, name='encoder_lstm'))

_, forward_h, forward_c, backward_h, backward_c = encoder_lstm(encoder_inputs)

state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

# Build the encoder network model
encoder_model = Model(inputs=encoder_inputs, outputs=[state_h, state_c],name='encoder')

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

SVG(model_to_dot(encoder_model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=encoder_model, show_shapes=False,
    to_file='encoder.pdf'
)

encoder_model.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None, 28)]   0           []                               
                                                                                                  
 bidirectional (Bidirectional)  [(None, 128),        47616       ['encoder_inputs[0][0]']         
                                 (None, 64),                                                      
                                 (None, 64),                                                      
                                 (None, 64),                                                      
                                 (None, 64)]                                                      
                                                                                            

In [ ]:
from keras.layers import Input, LSTM, Dense
from keras.models import Model

latent_dim =128

# inputs of the decoder network
decoder_input_h = Input(shape=(latent_dim,), name='decoder_input_h')
decoder_input_c = Input(shape=(latent_dim,), name='decoder_input_c')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# set the LSTM layer
decoder_lstm = LSTM(latent_dim, return_sequences=True, 
                    return_state=True, dropout=0.5, name='decoder_lstm')
decoder_lstm_outputs, state_h, state_c = decoder_lstm(decoder_input_x, 
                                                      initial_state=[decoder_input_h, decoder_input_c])

# set the dense layer
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_lstm_outputs)

# build the decoder network model
decoder_model = Model(inputs=[decoder_input_x, decoder_input_h, decoder_input_c],
                      outputs=[decoder_outputs, state_h, state_c],
                      name='decoder')

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

SVG(model_to_dot(decoder_model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=decoder_model, show_shapes=False,
    to_file='decoder.pdf'
)

decoder_model.summary()

Model: "decoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 decoder_input_x (InputLayer)   [(None, None, 30)]   0           []                               
                                                                                                  
 decoder_input_h (InputLayer)   [(None, 128)]        0           []                               
                                                                                                  
 decoder_input_c (InputLayer)   [(None, 128)]        0           []                               
                                                                                                  
 decoder_lstm (LSTM)            [(None, None, 128),  81408       ['decoder_input_x[0][0]',        
                                 (None, 128),                     'decoder_input_h[0][0]',  

In [ ]:
# input layers
encoder_input_x = Input(shape=(None, num_encoder_tokens), name='encoder_input_x')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# connect encoder to decoder
encoder_final_states = encoder_model([encoder_input_x])
decoder_lstm_output, _, _ = decoder_lstm(decoder_input_x, initial_state=encoder_final_states)
decoder_pred = decoder_dense(decoder_lstm_output)

model = Model(inputs=[encoder_input_x, decoder_input_x], 
              outputs=decoder_pred, 
              name='model_training')

In [ ]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

SVG(model_to_dot(model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=model, show_shapes=False,
    to_file='model_training.pdf'
)

model.summary()

Model: "model_training"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input_x (InputLayer)   [(None, None, 28)]   0           []                               
                                                                                                  
 decoder_input_x (InputLayer)   [(None, None, 30)]   0           []                               
                                                                                                  
 encoder (Functional)           [(None, 128),        47616       ['encoder_input_x[0][0]']        
                                 (None, 128)]                                                     
                                                                                                  
 decoder_lstm (LSTM)            [(None, None, 128),  81408       ['decoder_input_x[0]

In [ ]:
print('shape of encoder_input_data' + str(encoder_input_data.shape))
print('shape of decoder_input_data' + str(decoder_input_data.shape))
print('shape of decoder_target_data' + str(decoder_target_data.shape))

shape of encoder_input_data(20000, 18, 28)
shape of decoder_input_data(20000, 55, 30)
shape of decoder_target_data(20000, 55, 30)


In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
# use early stopping to exit training if validation loss is not decreasing even after certain epochs (patience)
earlystopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)

# save the best model with least validation loss
checkpointer = ModelCheckpoint(filepath="classifier-resnet-weights.hdf5", verbose=1, save_best_only=True)

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',metrics = ['accuracy'])

model.fit([encoder_input_data, decoder_input_data],  # training data
          decoder_target_data,                       # labels (left shift of the target sequences)
          batch_size=16, epochs=200, validation_split=0.2, callbacks=[checkpointer, earlystopping])

model.save('seq2seq.h5')

Epoch 1/200
 995/1000 [============================>.] - ETA: 0s - loss: 0.8029 - accuracy: 0.7646
Epoch 1: val_loss improved from inf to 0.76537, saving model to classifier-resnet-weights.hdf5
1000/1000 [==============================] - 16s 11ms/step - loss: 0.8025 - accuracy: 0.7647 - val_loss: 0.7654 - val_accuracy: 0.7634
Epoch 2/200
 996/1000 [============================>.] - ETA: 0s - loss: 0.6984 - accuracy: 0.7876
Epoch 2: val_loss improved from 0.76537 to 0.70672, saving model to classifier-resnet-weights.hdf5
1000/1000 [==============================] - 10s 10ms/step - loss: 0.6985 - accuracy: 0.7875 - val_loss: 0.7067 - val_accuracy: 0.7831
Epoch 3/200
 996/1000 [============================>.] - ETA: 0s - loss: 0.6716 - accuracy: 0.7951
Epoch 3: val_loss improved from 0.70672 to 0.67927, saving model to classifier-resnet-weights.hdf5
1000/1000 [==============================] - 10s 10ms/step - loss: 0.6715 - accuracy: 0.7951 - val_loss: 0.6793 - val_accuracy: 0.7888
Epoch

In [ ]:
# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [ ]:
import keras

def decode_sequence(input_seq, temperature = 0.2):
    states_value = encoder_model.predict(input_seq)

    target_seq = numpy.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)


        p = output_tokens[0, -1, :]
        p = numpy.asarray(p).astype('float64')
        p = p ** (1 / temperature)
        p = p / numpy.sum(p)

        next_onehot = numpy.random.multinomial(1,p,1)
        sampled_token_index = numpy.argmax(next_onehot)

        # To handle the 0 index error
        if sampled_token_index == 0:
          break
        
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        target_seq = numpy.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence


In [ ]:
for seq_index in range(2100, 2120):
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('English:       ', input_texts[seq_index])
    print('Spanish (true): ', target_texts[seq_index][1:-1])
    print('Spanish (pred): ', decoded_sentence[0:-1])


-
English:        hes skinny
Spanish (true):  el esta delgado
Spanish (pred):  el es a mantido
-
English:        hes strong
Spanish (true):  el es fuerte
Spanish (pred):  el esta bien
-
English:        hes stupid
Spanish (true):  el es estupido
Spanish (pred):  el es a resante
-
English:        hes stupid
Spanish (true):  no le llega agua al tanque
Spanish (pred):  el es a cansado
-
English:        hes stupid
Spanish (true):  es un salame
Spanish (pred):  el es a reste
-
English:        help me out
Spanish (true):  ayudame
Spanish (pred):  ayudame a mary
-
English:        help me out
Spanish (true):  ayudame a salir
Spanish (pred):  ayudame a masa
-
English:        help me out
Spanish (true):  echeme la mano
Spanish (pred):  ayudame a masir
-
English:        help me out
Spanish (true):  ayudame a salir
Spanish (pred):  ayudame a mary
-
English:        here i come
Spanish (true):  aqui vengo
Spanish (pred):  aqui estan los tios
-
English:        here i come
Spanish (true):  ya estoy aqu

In [ ]:
input_sentence = 'I love you'

tokenizer = Tokenizer(char_level=True, filters='')
tokenizer.word_index = input_token_index
seqs = tokenizer.texts_to_sequences([input_sentence])

input_sequence = pad_sequences(seqs, maxlen=max_encoder_seq_length, padding='post')

input_x = onehot_encode(input_sequence,max_encoder_seq_length,num_encoder_tokens )

translated_sentence = decode_sequence(input_x)

print('source sentence is: ' + input_sentence)
print('translated sentence is: ' + translated_sentence)

source sentence is: I love you
translated sentence is: me gusta



In [ ]:
clean_pairs = clean_data(pairs)[0:120000, :]
input_texts = clean_pairs[:, 0]
target_texts = ['\t' + text + '\n' for text in clean_pairs[:, 1]]

print('Length of input_texts:  ' + str(input_texts.shape))
print('Length of target_texts: ' + str(input_texts.shape))

Length of input_texts:  (120000,)
Length of target_texts: (120000,)


In [ ]:
max_encoder_seq_length = max(len(line) for line in input_texts)
max_decoder_seq_length = max(len(line) for line in target_texts)

print('max length of input  sentences: %d' % (max_encoder_seq_length))
print('max length of target sentences: %d' % (max_decoder_seq_length))

max length of input  sentences: 44
max length of target sentences: 87


In [ ]:
encoder_input_seq, input_token_index = text2sequences(max_encoder_seq_length, 
                                                      input_texts)
decoder_input_seq, target_token_index = text2sequences(max_decoder_seq_length, 
                                                       target_texts)

print('shape of encoder_input_seq: ' + str(encoder_input_seq.shape))
print('shape of input_token_index: ' + str(len(input_token_index)))
print('shape of decoder_input_seq: ' + str(decoder_input_seq.shape))
print('shape of target_token_index: ' + str(len(target_token_index)))

shape of encoder_input_seq: (120000, 44)
shape of input_token_index: 27
shape of decoder_input_seq: (120000, 87)
shape of target_token_index: 29


In [ ]:
num_encoder_tokens = len(input_token_index) + 1
num_decoder_tokens = len(target_token_index) + 1

print('num_encoder_tokens: ' + str(num_encoder_tokens))
print('num_decoder_tokens: ' + str(num_decoder_tokens))

num_encoder_tokens: 28
num_decoder_tokens: 30


In [ ]:
encoder_input_data = onehot_encode(encoder_input_seq, max_encoder_seq_length, num_encoder_tokens)
decoder_input_data = onehot_encode(decoder_input_seq, max_decoder_seq_length, num_decoder_tokens)

decoder_target_seq = numpy.zeros(decoder_input_seq.shape)
decoder_target_seq[:, 0:-1] = decoder_input_seq[:, 1:]
decoder_target_data = onehot_encode(decoder_target_seq, 
                                    max_decoder_seq_length, 
                                    num_decoder_tokens)

print(encoder_input_data.shape)
print(decoder_input_data.shape)
print(decoder_target_data.shape)

(120000, 44, 28)
(120000, 87, 30)
(120000, 87, 30)


In [ ]:
rand_indices = numpy.random.permutation(120000)
train_indices = rand_indices[0:96000]
valid_indices = rand_indices[96000:108000]
test_indices = rand_indices[108000:120000]


encoder_input_data_train = encoder_input_data[train_indices,:]
encoder_input_data_valid = encoder_input_data[valid_indices,:]
encoder_input_data_test = encoder_input_data[test_indices,:] 

decoder_input_data_train = decoder_input_data[train_indices,:]
decoder_input_data_valid = decoder_input_data[valid_indices,:]
decoder_input_data_test = decoder_input_data[test_indices,:] 


decoder_target_data_train = decoder_target_data[train_indices,:]
decoder_target_data_valid = decoder_target_data[valid_indices,:]
decoder_target_data_test = decoder_target_data[test_indices,:] 

print(encoder_input_data_train.shape)
print(encoder_input_data_valid.shape)
print(encoder_input_data_test.shape)
print(decoder_input_data_train.shape)
print(decoder_input_data_valid.shape)
print(decoder_input_data_test.shape)
print(decoder_target_data_train.shape)
print(decoder_target_data_valid.shape)
print(decoder_target_data_test.shape) 

(96000, 44, 28)
(12000, 44, 28)
(12000, 44, 28)
(96000, 87, 30)
(12000, 87, 30)
(12000, 87, 30)
(96000, 87, 30)
(12000, 87, 30)
(12000, 87, 30)


In [ ]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint, LearningRateScheduler
# use early stopping to exit training if validation loss is not decreasing even after certain epochs (patience)
earlystopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=20)

# save the best model with least validation loss
checkpointer = ModelCheckpoint(filepath="classifier-resnet-weights.hdf5", verbose=1, save_best_only=True)

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

model.fit([encoder_input_data_train, decoder_input_data_train],  
          decoder_target_data_train,                       
          batch_size=16, epochs=200, validation_data=([encoder_input_data_valid, decoder_input_data_valid], decoder_target_data_valid), callbacks=[checkpointer, earlystopping])

model.save('seq2seq_tune.h5')

Epoch 1/200
5997/6000 [============================>.] - ETA: 0s - loss: 0.7585
Epoch 1: val_loss improved from inf to 0.58264, saving model to classifier-resnet-weights.hdf5
6000/6000 [==============================] - 69s 11ms/step - loss: 0.7585 - val_loss: 0.5826
Epoch 2/200
5997/6000 [============================>.] - ETA: 0s - loss: 0.6956
Epoch 2: val_loss improved from 0.58264 to 0.54700, saving model to classifier-resnet-weights.hdf5
6000/6000 [==============================] - 66s 11ms/step - loss: 0.6956 - val_loss: 0.5470
Epoch 3/200
5997/6000 [============================>.] - ETA: 0s - loss: 0.6774
Epoch 3: val_loss improved from 0.54700 to 0.52728, saving model to classifier-resnet-weights.hdf5
6000/6000 [==============================] - 65s 11ms/step - loss: 0.6774 - val_loss: 0.5273
Epoch 4/200
5999/6000 [============================>.] - ETA: 0s - loss: 0.6669
Epoch 4: val_loss improved from 0.52728 to 0.51723, saving model to classifier-resnet-weights.hdf5
6000/6000

In [ ]:
input_test_texts = input_texts[test_indices]
target_texts = numpy.array(target_texts)
target_test_texts = target_texts[test_indices]

predicted_sentences = []


for seq_index in range(2100, 2800):
    input_seq = encoder_input_data_test[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    predicted_sentences.append(decoded_sentence[0:-1])
    print('-')
    print('English:       ', input_test_texts[seq_index])
    print('Spanish (true): ', target_test_texts[seq_index][1:-1])
    print('Spanish (pred): ', decoded_sentence[0:-1])

-
English:        we cant give up now
Spanish (true):  no nos podemos dar por vencidos ahora
Spanish (pred):  	a pieues se
-
English:        we thought youd been captured by the enemy
Spanish (true):  pensamos que habias sido capturado por el enemigo
Spanish (pred):  uese
-
English:        i no longer live in that neighborhood
Spanish (true):  ya no vivo en aquel barrio
Spanish (pred):  	a de orey
-
English:        tom committed suicide when he was thirty
Spanish (true):  tom se suicido a los treinta
Spanish (pred):  tad se p
-
English:        the loss amounts to a million dollars
Spanish (true):  el deficit se eleva a un millon de dolares
Spanish (pred):  er paca ue ra estobo e	 ro cadn	o ue dos ues ues
-
English:        you sound tired
Spanish (true):  pareces cansado
Spanish (pred):  pa
-
English:        crime does not pay
Spanish (true):  la delincuencia no compensa
Spanish (pred):  ras pa
-
English:        shes at home taking care of the kids
Spanish (true):  ella esta cuidando de

In [ ]:
target_list = [sentence.split() for sentence in target_test_texts[2100:2800]]
predicted_list = [sentence.split() for sentence in predicted_sentences]

In [ ]:
# Compute the BLEU score
from nltk.translate.bleu_score import corpus_bleu

references = target_list
candidates = predicted_list
score = corpus_bleu(references, candidates)
print(score)

1.6821193270375592e-155


/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
